In [27]:
import numpy as np
import scipy as sp

# Define known parameters
rho1 = 1600; mu1 = 40e-3; N1 = 28000; D1 = 60e-3; M1 = 0.600; P1 = 26e5
rho2 = 1159.5; mu2 = 9.5592e-3
g = 9.81  # Acceleration due to gravity in m/s^2

Q1 = M1/rho1

# Define lower and upper bounds
lb = [2000, 30e-3, 0.1, 1e5]  # Lower bounds
ub = [10000, 150e-3, 0.6, 8e5]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]
np.random.seed(100)

# Define the objective function (sum of squared residuals)
def pump_equations(x):
    N2, D2, M2, P2 = x

    # Specific speed equation residual
    F1 = (N1 * np.sqrt(M1/rho1) / (g * P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (g * P2/(rho2*g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = (M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = (g * P1/(rho1*g) / (N1**2 * D1**2)) - (g * P2/(rho2*g) / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = (rho1 * N1 * D1**2 / mu1) - (rho2 * N2 * D2**2 / mu2)
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F1, F2, F3, F4]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1, 1.0, 1.0, 1]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = pump_equations(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns
x0 = np.array([7500, 40e-3, 0.5, 8e5])

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
#result = sp.optimize.minimize(objective_function, x0, bounds=bounds, method='trust-constr', options={'disp': True, 'maxiter': 10000, 'xtol': 1e-16, 'gtol': 1e-16})
result = sp.optimize.differential_evolution(objective_function, bounds, disp = True, popsize=300, tol = 1e-16, maxiter = 1000)
#result = sp.optimize.root(pump_equations, x0, method='trust-constr', bounds=bounds, options={'xtol': 1e-10, 'maxiter': 2000})

differential_evolution step 1: f(x)= 383096.6577959282
differential_evolution step 2: f(x)= 383096.6577959282
differential_evolution step 3: f(x)= 383096.6577959282
differential_evolution step 4: f(x)= 383096.6577959282
differential_evolution step 5: f(x)= 383096.6577959282
differential_evolution step 6: f(x)= 383096.6577959282
differential_evolution step 7: f(x)= 72300.56921191406
differential_evolution step 8: f(x)= 72300.56921191406
differential_evolution step 9: f(x)= 72300.56921191406
differential_evolution step 10: f(x)= 11175.700036623091
differential_evolution step 11: f(x)= 11175.700036623091
differential_evolution step 12: f(x)= 655.8254568538723
differential_evolution step 13: f(x)= 655.8254568538723
differential_evolution step 14: f(x)= 140.61257960029621
differential_evolution step 15: f(x)= 140.61257960029621
differential_evolution step 16: f(x)= 140.61257960029621
differential_evolution step 17: f(x)= 140.61257960029621
differential_evolution step 18: f(x)= 140.612579600

In [28]:
N2, D2, M2, P2 = result.x[0], result.x[1], result.x[2], result.x[3]
Q2 = M2/rho2
specific_speed_1 = N1*np.sqrt(M1/rho1)/(P1/(rho1*g))**(3/4)
specific_speed_2 = N2*np.sqrt(M2/rho2)/(P2/(rho2*g))**(3/4)
print(f"Ns = {specific_speed_2:.4f}, Matching = {100-abs(specific_speed_1-specific_speed_2)/specific_speed_1*100:.4f} %")

flow_coeff_1 = Q1/N1/D1**3
flow_coeff_2 = Q2/N2/D2**3
print(f"Flow Coefficient = {flow_coeff_2:.4f}, Matching = {100-abs(flow_coeff_1-flow_coeff_2)/flow_coeff_1*100:.4f} %")

head_coeff_1 = g*P1/(rho1*g)/N1**2/D1**2
head_coeff_2 = g*P2/(rho2*g)/N2**2/D2**2
print(f"Head Coefficient = {head_coeff_2}, Matching = {100-abs(head_coeff_1-head_coeff_2)/head_coeff_1*100} %")

Reysnold_1 = rho1*N1*D1**2/mu1
Reysnold_2 = rho2*N2*D2**2/mu2
print(f"Reysnold Number = {Reysnold_2}, Matching = {100-abs(Reysnold_2-Reysnold_2)/Reysnold_1*100} %")

print(f"\nModel's Shaft Speed is {N2:.2f} RPM\nModel's Impeller Diameter is {D2*1e3:.2f} mm\nModel's Mass Flow Rate is {M2:.2f} Kg/s\nModel's Pressure Across Pump is {P2/1e5:.2f} Bar\nModel's Volumetric Flow Rate is {Q2*1000:.4f} L/s")

Ns = 11.7432, Matching = 100.0000 %
Flow Coefficient = 0.0001, Matching = 100.0000 %
Head Coefficient = 0.0005757511337868481, Matching = 100.0 %
Reysnold Number = 4032000.0, Matching = 100.0 %

Model's Shaft Speed is 7401.58 RPM
Model's Impeller Diameter is 67.02 mm
Model's Mass Flow Rate is 0.16 Kg/s
Model's Pressure Across Pump is 1.64 Bar
Model's Volumetric Flow Rate is 0.1381 L/s
